In [1]:
import numpy as np
import gvar as gv
import sys
sys.path.append('../')
#from h5glance import H5Glance 
import os
import lsqfit
import io
import yaml
import warnings  
warnings.simplefilter(action="default")

In [2]:
os.chdir('../../hyperon-xpt-master/xpt')

In [3]:
import fit_analysis as fa 
import fit_routine as fit
import i_o
import priors

/Users/grantdb/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/Users/grantdb/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88, got 80
  return f(*args, **kwds)
/Users/grantdb/anaconda3/lib/python3.7/site-packages/h5py/_hl/base.py:19: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import (Mapping, MutableMapping, KeysView,


In [4]:
import tests
import importlib
importlib.reload(tests)
importlib.reload(fit)
verbose_fitfcn = tests.VerboseFitfcn(force_gvars=True)
#print(verbose_fitfcn)

/Users/grantdb/anaconda3/lib/python3.7/site-packages/yaml/constructor.py:126: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  if not isinstance(key, collections.Hashable):


In [5]:
from importlib import reload
reload(i_o)
reload(fa)
reload(fit)

<module 'fit_routine' from '/Users/grantdb/LQCD/hyperon-xpt-master/xpt/fit_routine.py'>

In [6]:
file = '../data/hyperon_data.h5'
#H5Glance(file)
#file2 = '../data/input_data.h5'
#H5Glance(file2)
#file2 = '~/Documents/hyperon_ga/hyper_spec_axial.h5'

In [7]:
# a_fm = gv.load('a_fm_results.p')
# a_fm_out = gv.evalcorr([v for k,v in a_fm.items()])
# #print(a_fm_out)

In [8]:
# w0_a = gv.load('w0_a_results.p')
# w0_a_out = gv.evalcorr([v for k,v in w0_a.items()])
# print(w0_a_out)

In [9]:
from priors import prior
prior = prior
#print(prior)

In [10]:
f = open('models.yaml', 'r')
models = yaml.load(f)
    #print(keys)
model_info = {}
for item in models.values():
        model_info = item['proton:s_n2lo:d_n2lo:x_n2lo']
#print(model_info)

In [11]:
input_output = i_o.InputOutput()
data,ensembles = input_output.get_data(scheme='w0_imp')
prior = input_output.make_prior(data,prior)
phys_point_data = input_output.get_data_phys_point()

In [12]:
 fit_test = fit.fit_routine(prior=prior, data=data, model_info=model_info)

In [13]:
print(fit_test.fit)

/Users/grantdb/LQCD/hyperon-xpt-master/xpt/fit_routine.py:882: RuntimeWarning: invalid value encountered in log
  + p['b_{proton,4}']*xdata['lam_chi']*xdata['eps_pi']**4)
Least Square Fit:
  chi2/dof [dof] = 7.5 [17]    Q = 4.3e-19    logGBF = -126.5

Parameters:
             m_{xi,0}     1000 (1000)      [   1000 (1000) ]  
          m_{xi_st,0}     1300 (1000)      [   1300 (1000) ]  
         m_{lambda,0}     1000 (1000)      [   1000 (1000) ]  
          m_{sigma,0}     1200 (1000)      [   1200 (1000) ]  
       m_{sigma_st,0}     1400 (1000)      [   1400 (1000) ]  
          m_{omega,0}     1650 (1000)      [   1650 (1000) ]  
         m_{proton,0}        910 (10)      [    938 (1000) ]  
          m_{delta,0}      1229 (747)      [   1232 (1000) ]  
               s_{xi}     0.01 (5.00)      [     0.0 (5.0) ]  
           s_{xi,bar}     0.01 (5.00)      [     0.0 (5.0) ]  
           s_{lambda}     0.01 (5.00)      [     0.0 (5.0) ]  
        s_{sigma,bar}     0.01 (5.00)      

In [14]:
p = input_output.get_posterior(fit_test=fit_test,prior=prior,param=None)
fit_post = fit.fit_routine(prior=p, data=phys_point_data,model_info=model_info)
print(fit_post.fit)
#print(p)

/Users/grantdb/anaconda3/lib/python3.7/site-packages/lsqfit/__init__.py:912: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  D[:, i] = chivw_i.mdotder(self.cov)
/Users/grantdb/anaconda3/lib/python3.7/site-packages/lsqfit/__init__.py:1757: RuntimeWarning: divide by zero encountered in reciprocal
  data, wgts=-1, eps=eps, svdcut=svdcut, noise=noise[0],
/Users/grantdb/anaconda3/lib/python3.7/site-packages/lsqfit/_scipy.py:147: RuntimeWarning: invalid value encountered in multiply
  return numpy.asarray(f(x), float)


ValueError: Residuals are not finite in the initial point.

In [15]:
import mass_relations as mass 
reload(mass)

hyp_mass = mass.Mass_Combinations(lam= p['m_{lambda,0}'], sigma=p['m_{sigma,0}'], xi=p['m_{xi,0}'], delta=p['m_{xi_st,0}'], sigma_st=p['m_{sigma_st,0}'], xi_st=p['m_{xi_st,0}'], omega= p['m_{omega,0}'],p=prior )
#print(mass.m2, mass.m2_c)
print(hyp_mass.eps)
print('Label  Mass_combo')
for label, mass_combs in hyp_mass.M.items():
    print('{}    {}'.format(label, mass_combs))

print('c  Value')
for c, valuee in hyp_mass.c.items():
    print('{}    {}'.format(c, valuee))


print('R  Value')
for r, value in hyp_mass.R.items():
    print('{}    {}'.format(r, value))

print(mass.plot_baryon_mass_mpi)

[1.1893(67)e-07 1.6830(84)e-07 1.4738(57)e-07 1.1706(39)e-07 1.0504(44)e-07
 8.750(32)e-08 1.6501(66)e-07 1.4330(65)e-07 1.1673(47)e-07 1.0396(52)e-07
 8.794(34)e-08 1.7386(55)e-07 1.4632(40)e-07 1.2060(38)e-07 1.1911(46)e-07
 1.0822(35)e-07 9.128(34)e-08]
Label  Mass_combo
M_1    228(79)
M_2    46(16)
M_3    -1.1(1.6)
M_4    -0.03(32)
M_5    0.09(95)
M_6    0.1(1.1)
M_7    0.02(22)
M_8    0.003(32)
M_A    -0.002(141)
M_B    -0.0007(125)
M_C    -0.0002(87)
M_D    -0.002(16)
c  Value
c^{1,0}_0    0.43(15)
c^{1,0}_2    -0.114(40)
c^{8,0}_1    [-27136(38391) -19176(27130) -21898(30980) -27571(39006) -30725(43469)
 -36884(52181) -19559(27671) -22522(31864) -27649(39116) -31044(43919)
 -36698(51919) -18563(26263) -22057(31205) -26762(37862) -27097(38335)
 -29824(42193) -35358(50023)]
c^{8,0}_2    [10011(92107) 7074(65090) 8078(74328) 10171(93582) 11335(104290)
 13607(125193) 7215(66387) 8309(76447) 10200(93848) 11452(105371)
 13538(124564) 6848(63010) 8137(74866) 9873(90838) 9996(91974)
 11

AttributeError: module 'mass_relations' has no attribute 'plot_baryon_mass_mpi'

In [20]:
pdg = {}
pdg['lambda'] = input_output.get_data_phys_point(param='m_lambda')#gv.gvar(1115.683,0.006)
pdg['sigma'] = input_output.get_data_phys_point(param='m_sigma')#Gv.gvar(1192.642, 0.024)
pdg['sigma_st'] = input_output.get_data_phys_point(param='m_sigma_st')#gv.gvar(1382.80,0.35)
pdg['xi'] = input_output.get_data_phys_point(param='m_xi_st') #gv.gvar(1314.86,0.20)
pdg['xi_st'] = input_output.get_data_phys_point(param='m_xi_st')
pdg['delta'] = gv.gvar(1115.683,0.006)
pdg['omega'] = input_output.get_data_phys_point(param='m_omega')

In [21]:
# using pdg masses #
mass_pdg = mass.Mass_Combinations(lam= pdg['lambda'], sigma=pdg['sigma'], xi=pdg['xi'], delta=pdg['delta'], sigma_st=pdg['sigma_st'], xi_st=pdg['xi_st'],p=p )
#print(mass_pdg)
print('Label  Mass_combo')
for label, mass_combs in mass_pdg.M.items():
    print('{}    {}'.format(label, mass_combs))

Label  Mass_combo
M_1    194.096(17)
M_2    38.8191(34)
M_3    -4.29587(68)
M_4    -0.092944(35)
M_5    0.48740(27)
M_6    -0.51503(24)
M_7    -0.103007(48)
M_8    0.015605(16)
M_A    0.019141(62)
M_B    -0.0053808(26)
M_C    -0.0008227(21)
M_D    -0.0078026(78)


In [15]:
from importlib import reload
reload(fa)
fit_analysis = fa.fit_analysis(phys_point_data=phys_point_data, data=data,model_info=model_info, prior=prior)
hi = fit_analysis.error_budget
print(hi)

dict_values([1000(1000), 1200(1000), 1400(1000), 1000(1000), 1300(1000), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.3(3.0), 0.7(3.0), -0.5(3.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0(0), 0(0), 0(0), inf(0), 197.327(0), 1157.0(7.2), 134.80(30), 494.20(30), 1115.6830(60), 1193.154(27), 1384.57(39), 1318.28(11), 1533.40(34), 1672(29), 938(29), array([561.9(5.0), 493.9(4.0), 516.9(4.1), 544.9(4.4), 558.2(4.5),
       576.9(4.6), 494.1(3.6), 507.3(3.7), 530.1(3.9), 544.7(4.0),
       561.6(4.1), 508.1(3.4), 507.3(3.4), 530.4(3

In [16]:
hi = fit_analysis.error_budget
print(hi)

dict_values([1000(1000), 1200(1000), 1400(1000), 1000(1000), 1300(1000), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.3(3.0), 0.7(3.0), -0.5(3.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0(0), 0(0), 0(0), inf(0), 197.327(0), 1157.0(7.2), 134.80(30), 494.20(30), 1115.6830(60), 1193.154(27), 1384.57(39), 1318.28(11), 1533.40(34), 1672(29), 938(29), array([561.9(5.0), 493.9(4.0), 516.9(4.1), 544.9(4.4), 558.2(4.5),
       576.9(4.6), 494.1(3.6), 507.3(3.7), 530.1(3.9), 544.7(4.0),
       561.6(4.1), 508.1(3.4), 507.3(3.4), 530.4(3

In [17]:
sorted(hi.items())

AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
for j, ens_j in enumerate(fit_analysis.ensembles):
    print(j,ens_j)

In [29]:
reload(io)
m_proton = fit_analysis.fitfcn(p=p,data=phys_point_data,particle='proton')
m_pi = phys_point_data['m_pi']
print(m_pi)
#m_proton = assistant.extrapolate('proton')
#sigma = gv.gvar(m_pi)**2 *gv.deriv(gv.gvar(m_proton), gv.gvar(m_pi)**2)
sigma = m_pi**2 *gv.deriv(m_proton, m_pi**2)
print(sigma)

134.80(30)
35.56(16)


In [17]:
print(fit_analysis.posterior)

{'m_{xi,0}': 1000(1000), 'm_{xi_st,0}': 1300(1000), 'm_{lambda,0}': 1000(1000), 'm_{sigma,0}': 1200(1000), 'm_{sigma_st,0}': 1400(1000), 'm_{omega,0}': 1650(1000), 'm_{proton,0}': 910(10), 'm_{delta,0}': 1229(747), 's_{xi}': 0.01(5.00), 's_{xi,bar}': 0.01(5.00), 's_{lambda}': 0.01(5.00), 's_{sigma,bar}': 0.01(5.00), 's_{sigma}': 0.01(5.00), 's_{omega,bar}': 0.01(5.00), 'g_{xi,xi}': 0.3(3.0), 'g_{xi_st,xi}': 0.7(3.0), 'g_{xi_st,xi_st}': -0.5(3.0), 'g_{lambda,sigma}': 0.01(5.00), 'g_{lambda,sigma_st}': 0.01(5.00), 'g_{sigma,sigma}': 0.01(5.00), 'g_{sigma_st,sigma}': 0.01(5.00), 'g_{sigma_st,sigma_st}': 0.01(5.00), 'g_{proton,delta}': -5e-11 +- 5, 'g_{proton,proton}': 2e-13 +- 5, 'g_{delta,delta}': 0.01(5.00), 'a_{xi,4}': 0.01(5.00), 'a_{xi_st,4}': 0.01(5.00), 'b_{xi,4}': 0.01(5.00), 'b_{xi_st,4}': 0.01(5.00), 'a_{sigma,4}': 0.01(5.00), 'a_{sigma_st,4}': 0.01(5.00), 'b_{sigma,4}': 0.01(5.00), 'b_{sigma_st,4}': 0.01(5.00), 'a_{lambda,4}': 0.01(5.00), 'b_{lambda,4}': 0.01(5.00), 'a_{omega,4

In [20]:
print(fit_analysis._extrapolate_to_ens(ens=None,phys_params=None))

IndexError: index 17 is out of bounds for axis 0 with size 17

In [ ]:
m_pi = i_o.data_phys_point['m_pi']
m_proton = assistant.extrapolate('proton')
sigma = m_pi**2 *gv.deriv(m_proton, m_pi**2)

In [24]:
x = fit_analysis.fitter.y
print(x)


{'xi': array([1282(12), 1318(11), 1370(11), 1388(11), 1411(12), 1271.5(9.6),
       1283(10), 1318.6(9.9), 1336(10), 1358(10), 1258.0(8.6),
       1257.6(8.5), 1294.3(8.7), 1288.1(9.1), 1302.1(9.7), 1327.5(9.8)],
      dtype=object), 'xi_st': array([1527(16), 1563(19), 1620(21), 1631(27), 1671(15), 1535(13),
       1555(12), 1581(13), 1599(14), 1622(13), 1528(11), 1518(13),
       1557(10), 1554(11), 1555(13), 1584(13)], dtype=object), 'sigma': array([1201(10), 1215(13), 1291(11), 1312(12), 1347(12), 1186.3(9.6),
       1208.7(9.2), 1248.0(9.6), 1275.2(9.7), 1298(10), 1206(10),
       1194.5(8.9), 1236.8(8.3), 1229.1(9.2), 1237.6(9.5), 1280.5(9.0)],
      dtype=object), 'sigma_st': array([1423(23), 1461(23), 1541(26), 1573(22), 1601(17), 1462(14),
       1480(12), 1507(17), 1536(16), 1558(17), 1485(13), 1447(23),
       1499(10), 1499(11), 1490(21), 1537(13)], dtype=object), 'lambda': array([1062(26), 1142(11), 1229(10), 1260(10), 1304(11), 1090(11),
       1123.8(9.5), 1187.4(9.1), 12